In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import re

current_path = os.getcwd()
root_path = os.path.dirname(current_path)
root_path = os.path.dirname(root_path)
root_path = '/content/gdrive/Othercomputers/My Computer/Fire_risk_main/Fire_risk_main/data/Fire-data-paper-hongru_share/'
data_path= root_path + '/github_repo/data/'


# fire outcomes

In [ ]:

df_new=pd.read_pickle(data_path + 'df_all_events_with_outcome.pkl')

In [ ]:
city_zip_mapping=pd.read_excel(data_path + '/zip_cbsa_122022.xlsx',dtype=str,sheet_name='ZIP_CBSA_122022')
unique_zip =city_zip_mapping[['ZIP', 'CBSA','RES_RATIO']].drop_duplicates()
df_new = df_new.merge(unique_zip, left_on=['ZIP5'],right_on=['ZIP'], how='left')


In [ ]:
nan_0 = ['PROP_LOSS', 'PROP_VAL', 'CONT_LOSS', 'CONT_VAL','FF_INJ', 'FF_DEATH', 'OTH_INJ', 'OTH_DEATH']
df_new[nan_0] = df_new[nan_0].fillna(0)


df_new['total_loss'] = df_new['PROP_LOSS'].astype(int) + df_new['CONT_LOSS'].astype(int)


In [12]:
df_loss=df_new.copy()
df_loss = df_loss[df_loss['total_loss']>0]
threshold = df_loss['total_loss'].quantile(0.99)
df_loss = df_loss[df_loss['total_loss'] < threshold]

cpi_map={'2012':1,
         '2013':1.02,
         '2014':1.02,
         '2015':1.03,
         '2016':1.05,
         '2017':1.07,
         '2018':1.09,
         '2019':1.12,
         '2020':1.13,
         '2021':1.21,
         '2022':1.29
}
df_loss['total_loss'] = df_loss.apply(lambda row: row['total_loss'] / cpi_map.get(row['accident_year'], 1)*1.29, axis=1)

df_loss['total_loss']= df_loss['total_loss'].astype(int)* df_loss['RES_RATIO'].astype(float)

df_loss['num_event']=df_loss['RES_RATIO'].astype(float)

df_loss_grouped = df_loss.groupby('CBSA').agg({
    'num_event': 'sum',
    'total_loss': 'sum'
}).reset_index()

df_loss_grouped['ave_loss_per_event'] = df_loss_grouped['total_loss'] / df_loss_grouped['num_event']
df_loss_grouped['num_event'].sum()


np.float64(750772.9999999999)

In [37]:
def compute_cbsa_risk_proportion(df_consequence, consequence_type, city_zip_mapping):
    # Count number of events in each Risk_Group per ZIP5
    zip_risk_counts = df_consequence.groupby(['ZIP5', 'Risk_Group']).size().reset_index(name='count')

    # Merge ZIP-level counts with city-ZIP mapping
    zip_risk_props = zip_risk_counts.rename(columns={'ZIP5': 'ZIP'})
    zip_risk_with_cbsa = pd.merge(zip_risk_props, city_zip_mapping[['ZIP', 'CBSA', 'RES_RATIO']], on='ZIP', how='left')

    # Adjust counts using RES_RATIO
    zip_risk_with_cbsa['adj_count'] = zip_risk_with_cbsa['count'] * zip_risk_with_cbsa['RES_RATIO'].astype(float)

    # Aggregate adjusted count at CBSA-Risk_Group level
    cbsa_risk_props = zip_risk_with_cbsa.groupby(['CBSA', 'Risk_Group'])['adj_count'].sum().reset_index()

    # Total adjusted count per CBSA
    cbsa_total = cbsa_risk_props.groupby('CBSA')['adj_count'].sum().reset_index(name='total_adj_count')

    # Merge and compute proportion
    df_cbsa_risk = pd.merge(cbsa_risk_props, cbsa_total, on='CBSA')
    df_cbsa_risk['risk_proportion'] = df_cbsa_risk['adj_count'] / df_cbsa_risk['total_adj_count']
    
    # Add consequence type
    df_cbsa_risk['consequence_type'] = consequence_type
    
    return df_cbsa_risk

In [38]:
df_fs=df_new.copy()
df_fs = df_fs[df_fs['FIRE_SPRD'].notna()]
df_fs['Risk_Group'] = df_fs['FIRE_SPRD']

df_fs_grouped = compute_cbsa_risk_proportion(df_fs, 'spread', city_zip_mapping)

df_fs_grouped=df_fs_grouped[df_fs_grouped['Risk_Group']=='2']

In [ ]:
inj_specific=pd.read_pickle(data_path + 'inj_specific.pkl')
df_inj=df_new.copy()
df_inj=df_inj.merge(inj_specific, on='INCIDENT_KEY', how='inner')
vsl_proportion = [x * 1000 for x in [0.003, 0.047, 0.266, 0.593, 1]]

df_inj['vsl_proportion'] = (
    df_inj['SEV_1'] * vsl_proportion[0] +
    df_inj['SEV_2'] * vsl_proportion[1] +
    df_inj['SEV_3'] * vsl_proportion[2] +
    df_inj['SEV_4'] * vsl_proportion[3] +
    df_inj['SEV_5'] * vsl_proportion[4]
)

df_inj['num_event']=df_inj['RES_RATIO'].astype(float)

df_inj_grouped = df_inj.groupby('CBSA').agg({
    'num_event': 'sum',
    'vsl_proportion': 'sum'
}).reset_index()

df_inj_grouped['ave_vsl_per_event'] = df_inj_grouped['vsl_proportion'] / df_inj_grouped['num_event']
df_inj_grouped['ave_vsl_per_event']=df_inj_grouped['ave_vsl_per_event'].astype(float)
df_inj_grouped['num_event'].sum()


np.float64(48647.0)

In [40]:
df_inj_grouped.columns

Index(['CBSA', 'num_event', 'vsl_proportion', 'ave_vsl_per_event'], dtype='object')

# combine ignition rate and three consequence

In [45]:
final_fire_matrix=cbsa_summary.merge(df_fs_grouped[['CBSA','risk_proportion',]], on='CBSA', how='left')
final_fire_matrix=final_fire_matrix.merge(df_loss_grouped[['CBSA', 'num_event', 'total_loss', 'ave_loss_per_event']], on='CBSA', how='left')
final_fire_matrix=final_fire_matrix.merge(df_inj_grouped[['CBSA','num_event', 'vsl_proportion', 'ave_vsl_per_event']], on='CBSA', how='left',suffixes=('_loss','_inj'))
final_fire_matrix.rename(columns={'risk_proportion': 'fire_spread_2'}, inplace=True)
final_fire_matrix['fs_larger2']=1-final_fire_matrix['fire_spread_2']


# calculate ranking


In [ ]:
cbsa_county_mapping = pd.read_excel(data_path + '/list1_2023.xlsx', header=None, dtype=str)

# Set column names using the second row
cbsa_county_mapping.columns = cbsa_county_mapping.iloc[2]

# Drop the first two rows (original header and the new column names row)
cbsa_county_mapping = cbsa_county_mapping.drop(index=[0, 1,2]).reset_index(drop=True)

cbsa_name= cbsa_county_mapping[['CBSA Code', 'CBSA Title',
       'Metropolitan/Micropolitan Statistical Area',]].drop_duplicates()

In [ ]:
final_fire_matrix=final_fire_matrix.merge(cbsa_name, left_on='CBSA', right_on='CBSA Code', how='left')
final_fire_matrix['adj_population'].min()

In [ ]:

final_fire_matrix=final_fire_matrix[final_fire_matrix['adj_population'] >=100_000]
final_fire_matrix=final_fire_matrix.dropna()

final_fire_matrix = final_fire_matrix.reset_index(drop=True)

final_fire_matrix['rank_ign'] = final_fire_matrix['fire_rate_per_100_000_year'].rank(ascending=False, method='min')
final_fire_matrix['rank_ave_loss_per_event'] = final_fire_matrix['ave_loss_per_event'].rank(ascending=False, method='min')
final_fire_matrix['rank_ave_vsl_per_event'] = final_fire_matrix['ave_vsl_per_event'].rank(ascending=False, method='min')
final_fire_matrix['rank_fs_larger2'] = final_fire_matrix['fs_larger2'].rank(ascending=False, method='min')


num_rank=10


# smaller, higher risk
final_fire_matrix['pop_rank'] = (
    pd.qcut(
        final_fire_matrix['rank_pop'],
        q=num_rank,
        labels=False
    ).astype(int) + 1
)

final_fire_matrix['spread_rank'] = (
    pd.qcut(
        final_fire_matrix['rank_fs_larger2'],
        q=num_rank,
        labels=False
    ).astype(int) + 1
)

final_fire_matrix['rate_rank'] = (
    pd.qcut(
        final_fire_matrix['fire_rate_per_100_000_year'],
        q=num_rank,
        labels=False
    ).astype(int) + 1
)

final_fire_matrix['loss_rank'] = (
    pd.qcut(
        final_fire_matrix['rank_ave_loss_per_event'],
        q=num_rank,
        labels=False
    ).astype(int) + 1
)

final_fire_matrix['inj_rank'] = (
    pd.qcut(
        final_fire_matrix['rank_ave_vsl_per_event'],
        q=num_rank,
        labels=False
    ).astype(int) + 1       
)

final_fire_matrix['outcome_rank'] = final_fire_matrix['inj_rank'] + final_fire_matrix['spread_rank'] + final_fire_matrix['loss_rank']

# large value higher risk


epsilon = 1e-6  # to avoid zero

min_val = final_fire_matrix['outcome_rank'].min()
max_val = final_fire_matrix['outcome_rank'].max()

# Reverse scaling and avoid 0
final_fire_matrix['outcome_rank_scaled'] = (
    (max_val - final_fire_matrix['outcome_rank']) /
    (max_val - min_val + epsilon)
) * (1 - epsilon) + epsilon



epsilon = 1e-6  # to avoid zero

min_val = final_fire_matrix['rank_ign'].min()
max_val = final_fire_matrix['rank_ign'].max()

final_fire_matrix['ign_rank_scaled']  = (
    (max_val - final_fire_matrix['rank_ign']) /
    (max_val - min_val + epsilon)
) * (1 - epsilon) + epsilon

final_fire_matrix['total_fire_risk'] = final_fire_matrix['ign_rank_scaled']*final_fire_matrix['outcome_rank_scaled']

final_fire_matrix['overall_rank'] =  (
    pd.qcut(
        final_fire_matrix['total_fire_risk'],
        q=5,
        labels=False
    ).astype(int) + 1       
)